In [2]:
import chi

chi.use_site("CHI@UC")
chi.set("project_name", "CH-822636")
chi.set("image", "CC-CentOS7")
chi.set("keypair_name", "dimas-ccloud")

Now using CHI@UC:
URL: https://chi.uc.chameleoncloud.org
Location: Argonne National Laboratory, Lemont, Illinois, USA
Support contact: help@chameleoncloud.org


In [3]:
from chi import lease

res = []
lease.add_node_reservation(res, node_type="compute_skylake", count=2)
lease.add_fip_reservation(res, count=2)
start_date, end_date = lease.lease_duration(days=7)

l = lease.create_lease("ceph-benchmark", res, start_date=start_date, end_date=end_date)

print("Waiting for lease to start ...")

l = lease.wait_for_active(l["id"])

print("Lease started!")

reserved_fips = lease.get_reserved_floating_ips(l["id"])
print("ip 1 =",reserved_fips[0])
print("ip 1 =",reserved_fips[1])

Waiting for lease to start ...
Lease started!
ip 1 = 192.5.86.248
ip 1 = 192.5.87.174


In [4]:
from chi import server
reservation_id = ""
for item in l["reservations"]:
    if item["resource_type"] == "physical:host":
        reservation_id=item["id"]

instance = server.create_server(
    "ceph-instance", 
    reservation_id=reservation_id,
    count=2,
    image_name=chi.get("image")
)

print(instance[0])

print("Creating instance...")
chi.server.wait_for_active(instance[0].id, timeout=1200)
chi.server.wait_for_active(instance[1].id, timeout=1200)

print("Associating floating ip...")
ip1 = server.associate_floating_ip(instance[0].id, reserved_fips[0])
ip2 = server.associate_floating_ip(instance[1].id, reserved_fips[1])


server.wait_for_tcp(ip1, port=22)
server.wait_for_tcp(ip2, port=22)
print("Done!")

<Server: ceph-instance-2>
Creating instance...
Associating floating ip...
Done!


In [ ]:
# ip1="129.114.109.223"
# ip2="129.114.109.253"

In [5]:
from chi.ssh import Remote

r1 = Remote(ip=ip1)
r2 = Remote(ip=ip2)

In [7]:
local1=r1.run("hostname -I").stdout.strip()
local2=r2.run("hostname -I").stdout.strip()
# local1="10.52.2.185"
# local2="10.52.0.31"

10.140.82.58 
10.140.82.207 


/opt/conda/lib/python3.9/site-packages/paramiko/client.py:835: UserWarning: Unknown ssh-ed25519 host key for 192.5.87.174: b'2f329c4ffb0e77579326dea44f3679b3'
  warnings.warn(


In [8]:
r1.run('echo "Hello world!"')
r2.run('echo "Hello world!"')

Hello world!
Hello world!


<Result cmd='echo "Hello world!"' exited=0>

In [9]:
rList = [r1,r2]

# Setup loop devices

In [ ]:
for r in rList:
    r.run("sudo mkdir -p /mnt/extra")
    r.run("sudo chown cc /mnt/extra")
    r.run("mkdir -p /mnt/extra/loop-files/")

    # Create loop devices
    with r.cd("/mnt/extra/loop-files/"):
        r.run("sudo dd if=/dev/zero of=loopbackfile1.img bs=100M count=250")
        r.run("sudo dd if=/dev/zero of=loopbackfile2.img bs=100M count=250")
        r.run("sudo dd if=/dev/zero of=loopbackfile3.img bs=100M count=250")
        r.run("sudo dd if=/dev/zero of=loopbackfile4.img bs=100M count=250")
        r.run("sudo dd if=/dev/zero of=loopbackfile5.img bs=100M count=250")

        r.run("sudo losetup -fP loopbackfile1.img")
        r.run("sudo losetup -fP loopbackfile2.img")
        r.run("sudo losetup -fP loopbackfile3.img")
        r.run("sudo losetup -fP loopbackfile4.img")
        r.run("sudo losetup -fP loopbackfile5.img")
    print("Created loop devices")

    # Format loop devices
    r.run('printf "y" | sudo mkfs.ext4 /mnt/extra/loop-files/loopbackfile1.img')
    r.run('printf "y" | sudo mkfs.ext4 /mnt/extra/loop-files/loopbackfile2.img')
    r.run('printf "y" | sudo mkfs.ext4 /mnt/extra/loop-files/loopbackfile3.img')
    r.run('printf "y" | sudo mkfs.ext4 /mnt/extra/loop-files/loopbackfile4.img')
    r.run('printf "y" | sudo mkfs.ext4 /mnt/extra/loop-files/loopbackfile5.img')
    print("Formatted loop devices")

    # Mount loop devices
    r.run("mkdir -p /mnt/extra/loop-devs/loop0")
    r.run("mkdir -p /mnt/extra/loop-devs/loop1")
    r.run("mkdir -p /mnt/extra/loop-devs/loop2")
    r.run("mkdir -p /mnt/extra/loop-devs/loop3")
    r.run("mkdir -p /mnt/extra/loop-devs/loop4")

    with r.cd("/mnt/extra/loop-files/"):
        r.run("sudo mount -o loop /dev/loop0 /mnt/extra/loop-devs/loop0")
        r.run("sudo mount -o loop /dev/loop0 /mnt/extra/loop-devs/loop1")
        r.run("sudo mount -o loop /dev/loop0 /mnt/extra/loop-devs/loop2")
        r.run("sudo mount -o loop /dev/loop0 /mnt/extra/loop-devs/loop3")
        r.run("sudo mount -o loop /dev/loop0 /mnt/extra/loop-devs/loop4")
    print("Mounted loop devices")

250+0 records in
250+0 records out
26214400000 bytes (26 GB) copied, 18.8215 s, 1.4 GB/s
250+0 records in
250+0 records out
26214400000 bytes (26 GB) copied, 21.1548 s, 1.2 GB/s
250+0 records in
250+0 records out
26214400000 bytes (26 GB) copied, 22.5483 s, 1.2 GB/s
250+0 records in
250+0 records out
26214400000 bytes (26 GB) copied, 45.4053 s, 577 MB/s
250+0 records in
250+0 records out
26214400000 bytes (26 GB) copied, 55.4395 s, 473 MB/s
mke2fs 1.42.9 (28-Dec-2013)


Created loop devices
/mnt/extra/loop-files/loopbackfile1.img is not a block special device.
Proceed anyway? (y,n) Discarding device blocks:    4096/640000 528384/6400002625536/6400004722688/6400006295552/640000              done                            
Filesystem label=
OS type: Linux
Block size=4096 (log=2)
Fragment size=4096 (log=2)
Stride=0 blocks, Stripe width=0 blocks
1602496 inodes, 6400000 blocks
320000 blocks (5.00%) reserved for the super user
First data block=0
Maximum filesystem blocks=2155872256
196 block groups
32768 blocks per group, 32768 fragments per group
8176 inodes per group
Superblock backups stored on blocks: 
	32768, 98304, 163840, 229376, 294912, 819200, 884736, 1605632, 2654208, 
	4096000

Allocating group tables:   0/19      done                            
Writing inode tables:   0/19      done                            
Creating journal (32768 blocks): done
Writing superblocks and filesystem accounting information:   0/19      done



mke2fs 1.42.9 (28-Dec-2013)


/mnt/extra/loop-files/loopbackfile2.img is not a block special device.
Proceed anyway? (y,n) Discarding device blocks:    4096/6400002101248/6400004198400/6400006295552/640000              done                            
Filesystem label=
OS type: Linux
Block size=4096 (log=2)
Fragment size=4096 (log=2)
Stride=0 blocks, Stripe width=0 blocks
1602496 inodes, 6400000 blocks
320000 blocks (5.00%) reserved for the super user
First data block=0
Maximum filesystem blocks=2155872256
196 block groups
32768 blocks per group, 32768 fragments per group
8176 inodes per group
Superblock backups stored on blocks: 
	32768, 98304, 163840, 229376, 294912, 819200, 884736, 1605632, 2654208, 
	4096000

Allocating group tables:   0/19      done                            
Writing inode tables:   0/19      done                            
Creating journal (32768 blocks): done
Writing superblocks and filesystem accounting information:   0/19      

# Install Ceph

In [ ]:
for r in rList:
    r.run("sudo yum -y install subscription-manager")
    r.run("sudo yum -y install firewalld")
    # r.run("sudo subscription-manager repos --enable=rhel-7-server-extras-rpms")
    # r.run("sudo yum install -y https://dl.fedoraproject.org/pub/epel/epel-release-latest-7.noarch.rpm")
    
    # Add Ceph repo
    r.run('cat <<EOF | sudo tee /etc/yum.repos.d/ceph.repo \n[ceph-noarch] \nname=Ceph noarch packages \nbaseurl=https://download.ceph.com/rpm-luminous/el7/noarch \nenabled=1 \ngpgcheck=1 \ntype=rpm-md \ngpgkey=https://download.ceph.com/keys/release.asc \nEOF')
    r.run("sudo yum -y install ceph-deploy")
    
    # Setup config
    r.run("sudo yum -y install ntp ntpdate ntp-doc")
    # r.run("sudo ntpdate 0.us.pool.ntp.org")
    # r.run("sudo hwclock --systohc")
    # r.run("sudo systemctl enable ntpd.service")
    # r.run("sudo systemctl start ntpd.service")
    r.run("sudo yum -y install openssh-server")
    r.run(f'''sudo bash -c "echo {local1} node-1 >> /etc/hosts"''')
    r.run(f'''sudo bash -c "echo {local2} node-2 >> /etc/hosts"''')
    r.run("cat <<EOF | sudo tee ~/.ssh/config \nHost node-1 \n   Hostname node-1 \n   User cc \nHost node-2 \n   Hostname node-2 \n   User cc \nEOF")
    r.run("sudo ufw disable")
    # r.run("sudo setenforce 0")
    r.run("sudo sed -i 's/^SELINUX=enforcing$/SELINUX=disabled/' /etc/selinux/config")
    r.run("sudo yum -y install yum-plugin-priorities ")

# Deploy Ceph

1. SSH into the master machine (ip1)
2. Run (it will probably fail)
```
sudo mkdir -p /mnt/extra/ceph_cluster
sudo ceph-deploy new node-1 node-2
```
3. Run
```
sudo cat /root/.ssh/id_rsa.pub
```
Copy the public key
4. SSH into cc@\<ip2\>
5. Append ~/.ssh/authorized_keys with the copied public key
6. Run
```
sudo ceph-deploy new node-1 node-2
```

In [ ]:
for r in rList:
    r.run("sudo mv /etc/yum.repos.d/ceph.repo /etc/yum.repos.d/ceph-deploy.repo")

with r1.cd("/mnt/extra/ceph_cluster"):
    r1.sudo("ceph-deploy install --release luminous node-1 node-2")
    
for r in rList:
    r.run(f'''sudo sed -i "s|return TYPE == 'disk'|return TYPE == 'disk' or TYPE == 'loop' or TYPE == 'part'|g" /usr/lib/python2.7/site-packages/ceph_volume/util/disk.py''')
    r.run(f'''cat /usr/lib/python2.7/site-packages/ceph_volume/util/disk.py | grep "TYPE == 'loop'"''')

with r1.cd("/mnt/extra/ceph_cluster"):
    r1.sudo("ceph-deploy mon create-initial")
    r1.sudo("ceph-deploy admin node-1 node-2")
    r1.sudo("ceph-deploy mgr create node-1")
    r1.sudo("ceph-deploy mds create node-1 node-2")

for r in rList:
    r.run("sudo timedatectl set-ntp false")
    r.run("sudo ntpdate -u pool.ntp.org")
    r.run("sudo timedatectl set-ntp true")

with r1.cd("/mnt/extra/ceph_cluster"):
    r1.run("ceph-deploy mon create-initial")
    

# Create OSDs

In [ ]:
with r1.cd("/mnt/extra/ceph_cluster"):
    r1.sudo("ceph-deploy osd create --data /dev/loop1 node-1")
    r1.sudo("ceph-deploy osd create --data /dev/loop2 node-1")
    r1.sudo("ceph-deploy osd create --data /dev/loop3 node-1")
    r1.sudo("ceph-deploy osd create --data /dev/loop1 node-2")
    r1.sudo("ceph-deploy osd create --data /dev/loop2 node-2")
    r1.sudo("ceph-deploy osd create --data /dev/loop3 node-2")
    r1.run("sudo ceph osd pool create mypool 100 100 replicated")
    r1.run("sudo ceph osd pool application enable mypool rgw")
    r1.run("sudo ceph status")


# Ceph Dashboard

In [ ]:
with r1.cd("/mnt/extra/ceph_cluster"):
    r1.run("sudo ceph mgr module enable dashboard")
    r1.run("sudo ceph mgr services")

# Start Rados Gateway (RGW)

In [ ]:
r1.run("sudo yum -y install ceph-radosgw")
r1.run("sudo yum -y install radosgw-agent")
r1.run("sudo ceph-authtool --create-keyring /etc/ceph/ceph.client.radosgw.keyring")
r1.run("sudo chmod +r /etc/ceph/ceph.client.radosgw.keyring")
r1.run("sudo ceph-authtool /etc/ceph/ceph.client.radosgw.keyring -n client.radosgw.node-1 --gen-key")
r1.run("sudo ceph-authtool -n client.radosgw.node-1 --cap osd 'allow rwx' --cap mon 'allow rwx' /etc/ceph/ceph.client.radosgw.keyring")
r1.run("sudo ceph -k /etc/ceph/ceph.client.admin.keyring auth add client.radosgw.node-1 -i /etc/ceph/ceph.client.radosgw.keyring")
r1.run(f'''cat <<EOF | sudo tee -a /etc/ceph/ceph.conf \n[client.radosgw.node-1] \nhost = node-1 \nkeyring = /etc/ceph/ceph.client.radosgw.keyring \nrgw socket path = "" \nlog file = /var/log/radosgw/client.radosgw.node-1.log \nrgw frontends = "civetweb port=8080" \nrgw print continue = false \nEOF''')
r1.run("ceph-deploy --overwrite-conf config pull node-1")
r1.run("ceph-deploy --overwrite-conf config push node-2")
r1.run("sudo mkdir -p /var/lib/ceph/radosgw/ceph-radosgw.node-1")
r1.run("sudo mkdir -p /var/log/radosgw/")
r1.run("sudo chown cc /var/run/ceph")
r1.run("sudo chown cc /var/log/radosgw/")
r1.run("sudo systemctl start ceph-radosgw@radosgw.node-1")
r1.run("sudo systemctl status ceph-radosgw@radosgw.node-1")

r1.run("curl node-1:8080")


# Create S3 Credential

In [ ]:
r1.run(f'''sudo radosgw-admin user create --uid="testuser" --display-name="First User"''')
r1.run("sudo radosgw-admin user list")
r1.run("sudo radosgw-admin user info --uid=testuser")

In [ ]:
accessKey=""
secretKey=""

# Benchmark

## HSBench

In [ ]:
r1.run("sudo yum -y install jq")
r1.run("sudo yum install -y go")
r1.run("sudo go install github.com/aws/aws-sdk-go@latest")
r1.run("sudo go install github.com/markhpc/hsbench@latest ")

with r1.cd("/root/go/bin"):
    r1.sudo(f'''IP=`ifconfig | grep "inet 10" | awk '{print $2}'`''')
    r1.sudo("PORT=8080")
    r1.sudo(f'''access_key="${accessKey}"''')
    r1.sudo(f'''secret_key="${secretKey}"''')
    
    r1.sudo("./hsbench -a $access_key -s $secret_key -u http://$IP:$PORT  -z 4K -t 1 -n 2000 -b 1 -d -1 -m cxipgdcx -ri 10 -j test.log")
    r1.sudo("./hsbench -a $access_key -s $secret_key -u http://$IP:$PORT  -z 8K -t 1 -n 2000 -b 1 -d -1 -m cxipgdcx -ri 10 -j test.log")
    r1.sudo("./hsbench -a $access_key -s $secret_key -u http://$IP:$PORT  -z 16K -t 1 -n 2000 -b 1 -d -1 -m cxipgdcx -ri 10 -j test.log")
    r1.sudo("./hsbench -a $access_key -s $secret_key -u http://$IP:$PORT  -z 32K -t 1 -n 2000 -b 1 -d -1 -m cxipgdcx -ri 10 -j test.log")
    r1.sudo("./hsbench -a $access_key -s $secret_key -u http://$IP:$PORT  -z 64K -t 1 -n 2000 -b 1 -d -1 -m cxipgdcx -ri 10 -j test.log")
    r1.sudo("./hsbench -a $access_key -s $secret_key -u http://$IP:$PORT  -z 128K -t 1 -n 2000 -b 1 -d -1 -m cxipgdcx -ri 10 -j test.log")
    r1.sudo("./hsbench -a $access_key -s $secret_key -u http://$IP:$PORT  -z 256K -t 1 -n 2000 -b 1 -d -1 -m cxipgdcx -ri 10 -j test.log")
    r1.sudo("./hsbench -a $access_key -s $secret_key -u http://$IP:$PORT  -z 512K -t 1 -n 2000 -b 1 -d -1 -m cxipgdcx -ri 10 -j test.log")
    r1.sudo("./hsbench -a $access_key -s $secret_key -u http://$IP:$PORT  -z 1M -t 1 -n 2000 -b 1 -d -1 -m cxipgdcx -ri 10 -j test.log")
    r1.sudo("./hsbench -a $access_key -s $secret_key -u http://$IP:$PORT  -z 2M -t 1 -n 2000 -b 1 -d -1 -m cxipgdcx -ri 10 -j test.log")
    r1.sudo("./hsbench -a $access_key -s $secret_key -u http://$IP:$PORT  -z 4M -t 1 -n 2000 -b 1 -d -1 -m cxipgdcx -ri 10 -j test.log")
    
    r1.sudo("")


## Librados

In [ ]:
r1.run("sudo yum install librados2-devel")
r1.run("git clone https://github.com/dParikesit/ceph-benchmark.git")
with r1.cd("ceph-benchmark"):
    r1.run("g++ -std=c++11 -g -c ceph-client.cpp -o ceph-client.o")
    r1.run("g++ -std=c++11 -g ceph-client.o -lrados -o ceph-client")
    r1.run("sudo ./ceph-client")


# Use the API

Open 2 terminals

In the first terminal, run   
`kubectl port-forward pod/minio -n minio-dev 9000 9090`

In the second terminal do anything with these credential  
API endpoint = 127.0.0.1:9000  
Access key = 'minioadmin'  
Access secret = 'minioadmin'  